# Generate collection metadata

In [1]:
# TODO

# Generate item metadata

In [2]:
main_dir = 's3://sdap-dev-zarr/OCO3/outputs/veda'
target_dir = 'demo-2024.10.28-target/SIMULTEST_TFP_cog'

There also happens to be COGs in SIMULTEST_TFP_cog.

In [3]:
%%capture
!pip install apache-airflow rio_stac
#!pip install pyopenssl --upgrade

In [15]:
from pystac.utils import datetime_to_str

import rio_cogeo
import s3fs

In [7]:
s3fsfs = s3fs.S3FileSystem(anon=False)

In [8]:
files = s3fsfs.glob(f'{main_dir}/{target_dir}/oco3*')
len(files)

4990

In [9]:
first_file = f's3://{files[0]}'
first_file

's3://sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif'

In [41]:
!gdalinfo /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif


Driver: GTiff/GeoTIFF
Files: /vsis3/sdap-dev-zarr/OCO3/outputs/veda/demo-2024.10.28-target/SIMULTEST_TFP_cog/oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif
Size is 800, 800
Coordinate System is:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        MEMBER["World Geodetic System 1984 (G2296)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            A

In [10]:
rio_cogeo.cog_validate(first_file)

(True, [], [])

In [11]:
file_name = first_file.split("/")[-1]
file_name

'oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif'

In [ ]:
# git clone https://github.com/nasa-impact/veda-data-airflow
# mv veda-data-airflow veda_data_airflow

In [18]:
start_datetime, end_datetime = [datetime_to_str(d) for d in regex.extract_dates(first_file, datetime_range="day")[0:2]]
start_datetime, end_datetime

('2019-10-10T00:00:00Z', '2019-10-10T23:59:59Z')

In [19]:
event = events.RegexEvent(
    collection="oco3",
    item_id=file_name,
    title="OCO
    assets=dict(
        default=dict(href=first_file)
    ),
    start_datetime=start_datetime,
    end_datetime=end_datetime,
)

In [29]:
%load_ext autoreload
%autoreload 2
from veda_data_airflow.dags.veda_data_pipeline.utils.build_stac.utils import stac, regex, events

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
pystac_item = stac.generate_stac(event, testing=True)

/srv/conda/envs/notebook/lib/python3.12/site-packages/numpy/_core/_methods.py:188 RuntimeWarning: overflow encountered in multiply

/srv/conda/envs/notebook/lib/python3.12/site-packages/numpy/_core/_methods.py:199 RuntimeWarning: overflow encountered in reduce

In [38]:
pystac_item

<Item id=oco3-co2_cal001_2019-10-10T190139Z_filtered_xco2.tif>

In [40]:
import json
with open(f'{file_name.replace('.tif', '')}_item.json', 'w') as f:
    f.write(json.dumps(pystac_item.to_dict(), indent=2))